In [22]:
from google.colab import userdata
GITHUB_TOKEN=userdata.get('GITHUB_TOKEN')

In [3]:
!git lfs install
!git clone !git clone https://{GITHUB_TOKEN}@github.com/husn-app/Fashion.git
!apt-get install git-lfs
!cd Fashion

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [32]:
%%capture
pip install faiss-cpu open_clip_torch

In [52]:
import faiss, torch, statistics
import numpy as np
import open_clip
from tqdm import tqdm

torch.set_grad_enabled(False)

In [93]:
!pip3 freeze | grep faiss

faiss-cpu==1.8.0.post1


In [28]:
image_embeddings = torch.nn.functional.normalize(torch.load('Fashion/image_embeddings_kaggle.pt'))
index = faiss.IndexFlatIP(512)
index.add(image_embeddings.detach().numpy())

<ipython-input-28-e49f53986f9c>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  image_embeddings = torch.nn.functional.normalize(torch.load('Fashion/image_embeddings_kaggle

In [29]:
similarities, indexes = index.search(np.expand_dims(image_embeddings[0].detach().numpy(),axis=0),  25) # expects a batch. but latency seems to increase linearly with batch size.

In [31]:
queries = [
    'parachute jeans with pleats',
    'christmas dinner dress',
    'jumpsuit for diwali',
    'sexy red corset',
    'denim jeans with knee pockets',
    'Office wear',
    'Brunch outfit',
    'Feminine Lace Dress with Delicate Detailing',
    'Shirt',
    'kurta',
    'Basic comfy lounge shorts',
    'Boolywood sarees for wedding',
]

12

In [34]:
model, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/open_clip/factory.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=map_loc

In [36]:
tokens = tokenizer(queries)
query_embeddings = model.encode_text(tokens)

In [42]:
query_embeddings = torch.nn.functional.normalize(query_embeddings, dim=-1)

In [45]:
def getTopK(base_embedding, K=25):
    global image_embeddings
    probs = torch.nn.functional.cosine_similarity(image_embeddings, base_embedding.view(1, 512))
    topk_indices = probs.topk(K).indices
    return topk_indices, probs[topk_indices]

In [67]:
brute_results, brute_scores = [], []
for i in tqdm(range(len(queries))):
    results, scores = getTopK(query_embeddings[i], K=100)
    brute_results.append(results)
    brute_scores.append(scores)

100%|██████████| 12/12 [00:20<00:00,  1.72s/it]


In [73]:
faiss_scores, faiss_results = index.search(query_embeddings.detach().numpy(),  100)

In [89]:
for id in range(len(queries)):
    for i in range(100):
        if faiss_results[id][i] != brute_results[id][i].item():
            print(f'\nQuery {queries[id]} =====')
            print(f'Similar product mismatch: Brute: {brute_results[id][i].item()}, Faiss: {faiss_results[id][i]}, scores (brute,faiss): ({brute_scores[id][i].item()}, {faiss_scores[id][i].item()})')


Query christmas dinner dress =====
Similar product mismatch: Brute: 25795, Faiss: 57299, scores (brute,faiss): (0.29480957984924316, 0.29480960965156555)

Query christmas dinner dress =====
Similar product mismatch: Brute: 57299, Faiss: 25795, scores (brute,faiss): (0.29480957984924316, 0.29480960965156555)


In [94]:
for id in range(len(queries)):
    for i in range(100):
        if abs(faiss_scores[id][i] - brute_scores[id][i].item()) >  1e-6:
            print(f'\nQuery {queries[id]} =====')
            print(f'Similar product mismatch: Brute: {brute_results[id][i].item()}, Faiss: {faiss_results[id][i]}, scores (brute,faiss): ({brute_scores[id][i].item()}, {faiss_scores[id][i].item()})')